<a href="https://colab.research.google.com/github/fabarroso/MVP-ML/blob/main/mvpml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MVP -  Machine Learning
Nome: Fabio de Andrade Barroso

Matricula:4052025000158

Dataset original: https://basedosdados.org/dataset/dbd717cb-7da8-4efd-9162-951a71694541?table=a2e9f998-e2c2-49b7-858a-ae1daef46dc0

**Segurança no Estado de São Paulo - Dados estatísticos da Secretaria de Segurança Pública do Estado de São Paulo.**

O dataset contém informações sobre ocorrências policiais no estado de São Paulo, com diversas variáveis relacionadas a diferentes tipos de crimes (homicídios, furtos, roubos, estupros, etc.) por município, mês e ano.

**Organização:**
Governo de São Paulo

**Cobertura temporal:**
2002 - 2021

**1 - Objetivo**

Analisar o comportamento e a ocorrência de crimes ao longo do tempo, utilizando técnicas de Análise Exploratória de Dados (EDA), pré-processamento e aprendizado de máquina com modelos de regressão para identificar padrões, prever a quantidade de ocorrências criminais e compreender possíveis fatores que influenciam a variação desses indicadores.

**1.1 - Escopo**

Exploração dos dados: Investigar a distribuição, tendências temporais e correlações entre variáveis relacionadas a crimes (como homicídios, roubos, furtos, etc.).

Pré-processamento: Tratar valores ausentes, outliers e realizar transformações (normalização, codificação de variáveis categóricas e imputação) para preparar os dados para os modelos.

Modelagem preditiva (Regressão): Avaliar diferentes algoritmos de regressão (Linear, Random Forest, Gradient Boosting, XGBoost, LightGBM) com validação cruzada e busca de hiperparâmetros, para estimar o número de ocorrências criminais.

Avaliação: Comparar modelos com métricas de regressão (RMSE, MAE e R²) e interpretar variáveis de maior relevância (feature importance) no fenômeno estudado.

**1.2 - Contexto do Problema**

A segurança pública é um tema central em discussões políticas e sociais. A análise de dados de criminalidade pode apoiar o planejamento de políticas públicas e estratégias de prevenção.
Com o uso de modelos de regressão em machine learning, é possível estimar quantitativamente a ocorrência de crimes ao longo do tempo, identificar padrões sazonais, detectar tendências de crescimento ou queda e apontar variáveis que mais influenciam esses resultados. Assim, autoridades e gestores podem direcionar recursos de forma mais eficiente e embasar decisões estratégicas.

**2 - Ambiente**

As bibliotecas foram escolhidas para garantir um fluxo eficiente de análise de dados e modelagem. *Pandas* e *NumPy* são usadas para manipulação e cálculo dos dados. *Matplotlib* e *Seaborn* são utilizadas para visualização e exploração gráfica. *Scikit-learn*  facilita o pré-processamento, criação de modelos, validação cruzada e avaliação de desempenho. Essas ferramentas asseguram uma análise robusta e reprodutível do dataset.

Ao carregar o dataset e realizar experimentos com o modelo, a definição da *seed*   assegura que a divisão entre treino e teste e outras etapas aleatórias, como a inicialização do modelo, sejam consistentes. Isso é crucial para garantir que as avaliações de desempenho do modelo sejam justas e reproduzíveis em diferentes execuções.

Foi utilizado pd.set_option do Pandas, evitando assim que colunas ou textos ficassem truncados.

In [3]:
## Carga de bibliotecas utilizadas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import sys
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, learning_curve
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


# Definir o SEED para reprodutibilidade (controle de aleatoriedade)
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Ajustando as configurações para exibir todas as colunas e linhas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Exibição de resultados
print("Python:", sys.version.split()[0])
print("Seed global:", SEED)

Python: 3.12.11
Seed global: 42


**3 - Dados: carga, entendimento e qualidade**

O dataset foi carregado diretamente de uma URL no GitHub, utilizando pd.read_csv, com a configuração de delimitador para vírgula (,) e codificação adequada para caracteres especiais (ISO-8859-1).

In [4]:
##Carregamento dos Dados
# URL GitHub
url = 'https://raw.githubusercontent.com/fabarroso/dados_sp_gov_ssp/main/sp_gov_ssp.csv'

# Carregamento do dataset
df = pd.read_csv(url, delimiter=',', encoding='ISO-8859-1')

In [18]:
# Exibe o tamanho do dataset
print(f"Total de linhas: {df.shape[0]}")
print(f"Total de colunas: {df.shape[1]}")

print("\n" + "-"*655 + "\n")

# Exibe as primeiras linhas
print(df.head(10))

print("\n" + "-"*655 + "\n")

# Informações dos tipos de dados
df.info()

Total de linhas: 153876
Total de colunas: 27

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    ano  mes  id_municipio           regiao_ssp  homicidio_doloso  numero_de_vitimas_em_homicidio_doloso  homicidio_doloso_por_acidente_de_transito  numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito  homicidio_culposo_por_acidente_de_transito  homicidio_culposo_outros  tentativa_de_h

A exclusão dos últimos quatro meses do último ano pode ser justificada pelo fato de que esses meses estavam completamente vazios (ou seja, não continham registros válidos). Ao remover essas linhas vazias, observamos melhora na qualidade dos dados utilizados para análise, evitando que valores ausentes ou irrelevantes distorçam os resultados.

In [13]:
# Exibe as primeiras 10 linhas
print(df.head(10).to_string())

print("\n" + "-"*660 + "\n")

# Identificar o último ano no dataset
ultimo_ano = df['ano'].max()

# Identificar os últimos 4 meses desse último ano
meses_para_excluir = [9, 10, 11, 12]  # setembro, outubro, novembro, dezembro

# Criar um DataFrame com os dados que serão excluídos (últimos 4 meses do último ano)
df_eliminado = df[(df['ano'] == ultimo_ano) & (df['mes'].isin(meses_para_excluir))]

# Exibir as linhas eliminadas
print("Linhas Eliminadas:")
print(df_eliminado)

print("\n" + "-"*660 + "\n")

# Exibir a quantidade de linhas eliminadas
print("\nQuantidade de Linhas Eliminadas:", df_eliminado.shape[0])

# Filtrando o DataFrame para excluir os últimos 4 meses do último ano
df_filtrado = df[~((df['ano'] == ultimo_ano) & (df['mes'].isin(meses_para_excluir)))]

    ano  mes  id_municipio           regiao_ssp  homicidio_doloso  numero_de_vitimas_em_homicidio_doloso  homicidio_doloso_por_acidente_de_transito  numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito  homicidio_culposo_por_acidente_de_transito  homicidio_culposo_outros  tentativa_de_homicidio  lesao_corporal_seguida_de_morte  lesao_corporal_dolosa  lesao_corporal_culposa_por_acidente_de_transito  lesao_corporal_culposa_outras  latrocinio  numero_de_vitimas_em_latrocinio  total_de_estupro  estupro  estupro_de_vulneravel  total_de_roubo_outros  roubo_outros  roubo_de_veiculo  roubo_a_banco  roubo_de_carga  furto_outros  furto_de_veiculo
0  2002    1       3500105  Presidente Prudente               0.0                                    0.0                                        0.0                                                             0.0                                         0.0                       0.0                     0.0                              0.0       

**3.1 Análise exploratória (EDA)**

**4 - Pré Processamento**

**Conclusão**

A presente análise, desenvolvida a partir dos dados oficiais da Secretaria de Segurança Pública do Estado de São Paulo (SSP-SP), buscou compreender padrões criminais e aplicar técnicas de regressão em aprendizado de máquina para prever a ocorrência de homicídios dolosos. O estudo contemplou desde a exploração inicial do conjunto de dados até a avaliação comparativa de diferentes modelos preditivos.

Os resultados obtidos permitem destacar alguns pontos relevantes:

Padrões temporais e sazonais – A ocorrência de crimes apresentou flutuações significativas ao longo do tempo, evidenciando sazonalidade em determinados períodos do ano. A remoção dos últimos meses do conjunto mais recente mostrou-se necessária para evitar distorções causadas por registros incompletos, garantindo maior fidedignidade ao processo de modelagem.

Qualidade e preparação dos dados – O pré-processamento realizado (tratamento de valores ausentes, normalização, winsorização de outliers e codificação de variáveis categóricas) foi determinante para melhorar a consistência estatística dos dados e a robustez dos modelos de regressão aplicados.

Relações entre variáveis – A análise de correlação evidenciou vínculos entre diferentes categorias de crimes e os homicídios dolosos, sugerindo que fenômenos criminais não ocorrem de forma isolada, mas sim interconectada, refletindo dinâmicas sociais e territoriais complexas.

Desempenho dos modelos de regressão – A comparação entre algoritmos mostrou que os métodos baseados em árvores de decisão e boosting (Gradient Boosting, XGBoost e LightGBM) apresentaram resultados superiores em termos de capacidade explicativa (R²) e menores erros preditivos (RMSE e MAE), em relação a modelos lineares tradicionais.

Explicabilidade e aplicação prática – A análise da importância das variáveis reforçou a relevância de fatores temporais e contextuais na previsão de homicídios dolosos. Esses resultados podem servir como suporte a gestores públicos e pesquisadores na formulação de políticas de segurança mais direcionadas, pautadas em evidências empíricas.

De forma geral, conclui-se que a aplicação de técnicas de regressão em aprendizado de máquina é eficaz para o estudo de dados criminais, possibilitando tanto a previsão quantitativa de ocorrências quanto a identificação de fatores de maior impacto. Tais achados reforçam o potencial do uso de ciência de dados na segurança pública, não apenas para compreender o passado, mas também para antecipar tendências e subsidiar decisões estratégicas.

Por fim, cabe destacar que a análise está sujeita a limitações inerentes ao próprio dataset, como a possível subnotificação de ocorrências e a ausência de variáveis socioeconômicas ou demográficas, que poderiam ampliar a capacidade explicativa dos modelos. Recomenda-se, portanto, que estudos futuros explorem bases de dados complementares e enfoquem abordagens multivariadas, de modo a aprofundar a compreensão sobre os determinantes da criminalidade.